In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from os import listdir
from os.path import isfile, join
import glob
import requests

In [2]:
# load csv files collected from fec.gov website
fec_path = 'data/fec-gov'
fec_files = glob.glob(fec_path + "/*.csv")

# create a list of dataframes from the csv files
fec_list_df = []
for x in fec_files:
# set no header and skip first two rows, as we do not use these rows
    df = pd.read_csv(x, header=None, error_bad_lines=False, skiprows=2)
    fec_list_df.append(df)

# concatenate dataframes from fec.gov files into single fec_df
fec_df = pd.concat(fec_list_df, axis=0, ignore_index=True)

# create new dataframe with only the columns we need from fec_df
new_fec_df = fec_df[[0,6,7,8,9,12,13,14,15,16,19,23,24,17,20,21,26]].copy()
new_fec_df = new_fec_df.rename(columns = { 0: "schedule",
                             6: "pac",
                             7: "last_name",
                             8: "first_name",
                             9: "mi",
                             12: "address_1",
                             13: "address_2",
                             14: "city",
                             15: "state",
                             16: "zip_code",
                             19: "contrib_date",
                             23: "employer",
                             24: "occupation",
                             17: "contrib_cycle",
                             20: "contrib_itemize",
                             21: "contrib_agg",
                             26: "conduit"})

# keep only SA11AI (individual contributions) in our new_fec_df dataframe
new_fec_df = new_fec_df.query("schedule == 'SA11AI'")

# remove duplicate rows created by conduit contributions (mostly ActBlue)
new_fec_df = new_fec_df.loc[new_fec_df['conduit'].isnull()]

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,27,28,29,31,32,33,34,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,27,28,29,33,34,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
new_fec_df

,schedule,pac,last_name,first_name,mi,address_1,address_2,city,state,zip_code,contrib_date,employer,occupation,contrib_cycle,contrib_itemize,contrib_agg,conduit
0,SA11AI,NaN,Cobb,Craig,NaN,360 Nueces St,Apt 1513,Austin,TX,787014263,20180709,Allclear ID,Senior Software Engineer,G2018,100.0,925.00,NaN
2,SA11AI,NaN,Thomas,Mike,NaN,1400 Stratford Pl,NaN,McKinney,TX,750717481,20180709,Self Employed,Attorney,G2018,200.0,500.00,NaN
4,SA11AI,NaN,Ocon,Angel,NaN,2105 Normandy Dr,NaN,Irving,TX,750605617,20180709,Electrical Surplus Of Texas,Electrical Equipment Technician,G2018,63.0,176.00,NaN
6,SA11AI,NaN,Weiner,Claire,NaN,1415 Madrid Pl,NaN,Santa Fe,NM,875054641,20180709,Not Employed,Not Employed,G2018,50.0,525.00,NaN
8,SA11AI,NaN,Gleason,Kevin,NaN,240 N Highland Ave NE,Unit 2401,Atlanta,GA,303075622,20180719,Tanner Health,Physician,G2018,50.0,200.00,NaN
10,SA11AI,NaN,Stanley,Len,NaN,1108 N Mangum St,NaN,Durham,NC,277011932,20180719,Not Employed,Not Employed,G2018,25.0,262.50,NaN
12,SA11AI,NaN,Kulhavy,David,L,504 Mockingbird Ln,NaN,Nacogdoches,TX,759646522,20180719,Stephen F. Austin State University,Educator,G2018,25.0,790.00,NaN
14,SA11AI,NaN,White,Merri,NaN,6340 Bridle Trl,NaN,Caddo Mills,TX,751356291,20180730,Beacon Hill,Accountant,G2018,100.0,420.75,NaN
16,SA11AI,NaN,Ward,Bruce,NaN,2237 Pinehurst St,NaN,Tyler,TX,757035809,20180730,Not Employed,Not Employed,G2018,250.0,525.00,NaN
18,SA11AI,NaN,Goodwin,K.,NaN,3213 Brookshire Dr,NaN,Plano,TX,750754708,20180730,The Quatrefoil Group Inc.,CEO,G2018,100.0,300.00,NaN


In [4]:
# pull out 100 records to test with Whitepages API
sample_df = new_fec_df.iloc[0:2]

sample_df

,schedule,pac,last_name,first_name,mi,address_1,address_2,city,state,zip_code,contrib_date,employer,occupation,contrib_cycle,contrib_itemize,contrib_agg,conduit
0,SA11AI,NaN,Cobb,Craig,NaN,360 Nueces St,Apt 1513,Austin,TX,787014263,20180709,Allclear ID,Senior Software Engineer,G2018,100.0,925.0,NaN
2,SA11AI,NaN,Thomas,Mike,NaN,1400 Stratford Pl,NaN,McKinney,TX,750717481,20180709,Self Employed,Attorney,G2018,200.0,500.0,NaN


In [5]:
# limit columns to just what is needed for the API call 
sample_df = sample_df.iloc[:,5:10]
sample_df

,address_1,address_2,city,state,zip_code
0,360 Nueces St,Apt 1513,Austin,TX,787014263
2,1400 Stratford Pl,NaN,McKinney,TX,750717481


In [6]:
sample_df['zip_5'] = sample_df['zip_code'].astype(str).str[:5]
sample_df

,address_1,address_2,city,state,zip_code,zip_5
0,360 Nueces St,Apt 1513,Austin,TX,787014263,78701
2,1400 Stratford Pl,NaN,McKinney,TX,750717481,75071


In [7]:
# replace spaces with '+' for whitepages API format
formatted_sample_df = sample_df.replace({' ': '+'}, regex=True)
formatted_sample_df

,address_1,address_2,city,state,zip_code,zip_5
0,360+Nueces+St,Apt+1513,Austin,TX,787014263,78701
2,1400+Stratford+Pl,NaN,McKinney,TX,750717481,75071


In [8]:
# concatenate address columns into one list of addresses
formatted_sample_df['addresses'] = 'street_line_1=' + formatted_sample_df['address_1'].astype(str) \
                                    + '&city=' + formatted_sample_df['city'].astype(str) \
                                    + '&postal_code=' + formatted_sample_df['zip_5'].astype(str) \
                                    + '&state_code=' + formatted_sample_df['state'].astype(str) \
                                    + '&api_key=[api-key]'

formatted_sample_df

,address_1,address_2,city,state,zip_code,zip_5,addresses
0,360+Nueces+St,Apt+1513,Austin,TX,787014263,78701,street_line_1=360+Nueces+St&city=Austin&postal...
2,1400+Stratford+Pl,NaN,McKinney,TX,750717481,75071,street_line_1=1400+Stratford+Pl&city=McKinney&...


In [9]:
addresses = formatted_sample_df['addresses'].tolist()
addresses

['street_line_1=360+Nueces+St&city=Austin&postal_code=78701&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7',
 'street_line_1=1400+Stratford+Pl&city=McKinney&postal_code=75071&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7']

In [13]:
# Starting URL for Whitepages API Call
url = "https://proapi.whitepages.com/3.0/location?" 

# List of donor address data


# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, address in enumerate(addresses):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    address_url = url + address
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, address))

    # Add 1 to the record count
    record_count += 1

    print(address_url)
        
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        address_search = requests.get(address_url).json()
        print(type(address_search))
        phone = address_search['current_residents'][0]['phones'][0]['phone_number']
        address_data ={
            "Phone": phone
            }

    # If an error is experienced, skip the city
    except:
        print("Address not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | street_line_1=360+Nueces+St&city=Austin&postal_code=78701&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7
https://proapi.whitepages.com/3.0/location?street_line_1=360+Nueces+St&city=Austin&postal_code=78701&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7
<class 'dict'>
21
Address not found. Skipping...
Processing Record 2 of Set 1 | street_line_1=1400+Stratford+Pl&city=McKinney&postal_code=75071&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7
https://proapi.whitepages.com/3.0/location?street_line_1=1400+Stratford+Pl&city=McKinney&postal_code=75071&state_code=TX&api_key=798a9277ff2440b1863f73adc5dbf7b7
<class 'dict'>
21
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
address_data

{'Phone': '+18067959992'}

In [21]:
address_search

{'id': 'Location.0b95540e-e70d-4666-b61a-dbcb91c0eeab',
 'is_valid': True,
 'street_line_1': '1400 Stratford Pl',
 'street_line_2': None,
 'city': 'McKinney',
 'postal_code': '75071',
 'zip4': '7481',
 'state_code': 'TX',
 'country_code': 'US',
 'lat_long': {'latitude': 33.213913,
  'longitude': -96.72142,
  'accuracy': 'RoofTop'},
 'is_active': True,
 'is_commercial': False,
 'is_forwarder': False,
 'delivery_point': 'SingleUnit',
 'current_residents': [{'id': 'Person.a9bb57a2-8c1f-4017-b6db-1a6b13c054ad',
   'name': 'Laurel Owens Burnette',
   'firstname': 'Laurel',
   'middlename': 'Owens',
   'lastname': 'Burnette',
   'alternate_names': [],
   'age_range': '85-89',
   'gender': 'Female',
   'type': 'Person',
   'industry': None,
   'link_to_address_start_date': '2017-11-01',
   'phones': [{'id': 'Phone.99716fef-a2e1-4b08-cfe3-bc7128b7c1c6',
     'phone_number': '+18067959992',
     'line_type': 'Landline'}],
   'historical_addresses': [{'id': 'Location.70874516-c179-4226-8329-2461

In [30]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [31]:
flatten_dict = flatten(address_search)

In [34]:
flatten_dict

{'id': 'Location.0b95540e-e70d-4666-b61a-dbcb91c0eeab',
 'is_valid': True,
 'street_line_1': '1400 Stratford Pl',
 'street_line_2': None,
 'city': 'McKinney',
 'postal_code': '75071',
 'zip4': '7481',
 'state_code': 'TX',
 'country_code': 'US',
 'lat_long_latitude': 33.213913,
 'lat_long_longitude': -96.72142,
 'lat_long_accuracy': 'RoofTop',
 'is_active': True,
 'is_commercial': False,
 'is_forwarder': False,
 'delivery_point': 'SingleUnit',
 'current_residents': [{'id': 'Person.a9bb57a2-8c1f-4017-b6db-1a6b13c054ad',
   'name': 'Laurel Owens Burnette',
   'firstname': 'Laurel',
   'middlename': 'Owens',
   'lastname': 'Burnette',
   'alternate_names': [],
   'age_range': '85-89',
   'gender': 'Female',
   'type': 'Person',
   'industry': None,
   'link_to_address_start_date': '2017-11-01',
   'phones': [{'id': 'Phone.99716fef-a2e1-4b08-cfe3-bc7128b7c1c6',
     'phone_number': '+18067959992',
     'line_type': 'Landline'}],
   'historical_addresses': [{'id': 'Location.70874516-c179-422